<a href="https://colab.research.google.com/github/emma-00/CNN-Learn/blob/main/CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install torch torchvision matplotlib numpy pandas

**<h1>*`Step 1: Setup and Imports`*</h1>**


**1. Import the necessary libraries:**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np


**<h1>*`Step 2: Load and Explore the MNIST Dataset`*</h1>**


**1. Define transformations to normalize the data:**

Transformations are used to preprocess the dataset.

The ToTensor transformation converts image data (which is typically in the form of a NumPy array or PIL image) into a PyTorch tensor.

Normalize scales pixel values from the range [0, 255] to [0, 1]

In PyTorch, normalization is applied using the formula:

$$
\text{Normalized Value} = \frac{\text{Value} - \text{Mean}}{\text{Standard Deviation}}
$$

Values centered around 0 reduce difficulties with a range (-1, 1) in gradient computations, increases stability.


In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


**Creating datasets:**

In [ ]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)


Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.30MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 28.9k/28.9k [00:00<00:00, 154kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



**3. Create DataLoaders for batch processing:**

DataLoader handles batch processing by grouping data samples into batches of the specified size. It also shuffles the data if required.

**Why?**

* Neural networks often process large datasets. Loading the entire dataset into memory is impractical.

* Computing gradients for the entire dataset in a single forward and backward pass is computationally expensive and slow.

* If a single sample is used for gradient updates, it can lead to noisy updates that overfit the training data.

* Modern hardware (like GPUs) is optimized for parallel computations.By using batches, multiple samples can be processed simultaneously, taking full advantage of the hardware.

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

**4. Visualize a few training samples:**

We visualize a few samples to confirm that the data is loaded correctly.


In [ ]:
def visualize_samples(data_loader):
    data_iter = iter(data_loader)
    images, labels = next(data_iter)
    images = images[:8]

    plt.figure(figsize=(10, 4))
    for i in range(8):
        plt.subplot(1, 8, i + 1)
        plt.imshow(images[i][0], cmap='gray')
        plt.title(f"Label: {labels[i]}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

visualize_samples(train_loader)


**<h1>*`Step 3: Define the CNN Model`*</h1>**


**1. Build a simple CNN:**

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # Input layer to hidden layer
        self.fc2 = nn.Linear(128, 10)       # Hidden layer to output layer

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the input image
        x = F.relu(self.fc1(x))  # Apply ReLU activation
        x = self.fc2(x)          # Output layer
        return F.log_softmax(x, dim=1)  # Apply log-softmax for classification


**<h1>*`Step 4: Train the CNN`*</h1>**


**1. Initialize the model, loss function, and optimizer:**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


**2. Define the training loop:**


In [ ]:
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")


**3. Train the model:**

In [ ]:
train_model(model, train_loader, criterion, optimizer, epochs=5)


**<h1>*`Step 5: Evaluate the Model`*</h1>**


**1. Define the evaluation loop:**

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

evaluate_model(model, test_loader)


**<h1>*`Step 6: Visualize Predictions`*</h1>**


**1. Visualize predictions on test data:**

In [ ]:
def visualize_predictions(model, test_loader):
    model.eval()
    data_iter = iter(test_loader)
    images, labels = next(data_iter)
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

    plt.figure(figsize=(10, 4))
    for i in range(8):
        plt.subplot(1, 8, i + 1)
        plt.imshow(images[i].cpu().squeeze(), cmap='gray')
        plt.title(f"P: {predicted[i].item()}\nT: {labels[i].item()}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

visualize_predictions(model, test_loader)
